In [36]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import subprocess

In [50]:
## Function to find event files/lc files from the directory 
def find_event_files(base_directory):
    event_files = []
    
    # Regular expression to match the pattern ObsID/xti/event_cl/
    obsid_pattern = re.compile(r'^\d{10}$')
    #obsid_pattern = re.compile(r'^\d{10}/xti/event_cl/$')
    #For one file
    #obsid_pattern = re.compile(r'6050390244/xti/event_cl/$')
    
    # Regular expressions to match the specific filename patterns
    file_pattern = re.compile(r'^ni\d{10}_0mpu7_cl_(night|daynotconsidered)_barycorr_orbit_piexpiex_yes.evt$')
    #file_pattern = re.compile(r'^ni\d{10}_cl_(night|day)mpu7_sr_night_barycorr.lc$')


    for obsid_dir in os.listdir(base_directory):
        obsid_path = os.path.join(base_directory, obsid_dir)

        # Check if it's a directory and matches the 10-digit ObsID pattern
        if os.path.isdir(obsid_path) and obsid_pattern.match(obsid_dir):
            #print(f"Found 10-digit ObsID folder: {obsid_dir}")

            # Look for files in this directory matching the .evt pattern
            for file in os.listdir(obsid_path):
                if file_pattern.match(file):
                    full_path = os.path.join(obsid_path, file)
                    event_files.append(full_path)
                    # print(f"Found .evt file: {full_path}")

    return event_files

In [ ]:
base_directory='./reduced_output'

test_input="/home/supremekai/thesis/J0243_6p6124/nicer/reduced_output/6050390231/ni6050390231_0mpu7_cl_night_barycorr_orbit_piexpiex_yes.evt"
test_output="/home/supremekai/thesis/J0243_6p6124/nicer/reduced_output/6050390231/ni6050390231_0mpu7_cl_night_barycorr_orbit_piexpiex_yes_trial_phase_resolved_test.evt"

#Running the code to find event files
first_input_orbit_file_list=find_event_files(base_directory)
print("Total number of input event files found: ",len(first_input_orbit_file_list))
for first_input_file in first_input_orbit_file_list:
    print("*******Adding Orbit Time from common reference******")
    print("Working on the file: ",first_input_file)
    print(first_input_file.split("/")[2])
    first_output_file = first_input_file.replace(".evt", "_phase_added.evt")
    first_ftcalc_command=f'ftcalc infile={first_input_file+"[events]"} outfile={first_output_file} column=ORBIT_REF_TIME expression="(ORBITTIME-(125979582.800))" clobber=yes'
    try:
        subprocess.run(first_ftcalc_command,shell=True, check=True)
        print("Successfully saved as: ",first_output_file)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {first_input_file}: {e}")
    

    print("*******Adding Phase column in same file******")
    second_input_file=first_output_file
    second_output_file=first_output_file
    second_ftcalc_command=f'ftcalc infile={second_input_file+"[events]"} outfile={second_output_file} column=PHASE expression="(0.102035610428039*(ORBIT_REF_TIME))%1" clobber=YES'                 
    
    try:
        subprocess.run(second_ftcalc_command,shell=True, check=True)
        print("Successfully saved as: ",second_output_file)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {first_input_file}: {e}")
    break



Total number of input event files found:  67
*******Adding Orbit Time from common reference******
Working on the file:  ./reduced_output/6050390261/ni6050390261_0mpu7_cl_night_barycorr_orbit_piexpiex_yes.evt
6050390261
Successfully saved as:  ./reduced_output/6050390261/ni6050390261_0mpu7_cl_night_barycorr_orbit_piexpiex_yes_phase_added.evt
*******Adding Phase column in same file******
Successfully saved as:  ./reduced_output/6050390261/ni6050390261_0mpu7_cl_night_barycorr_orbit_piexpiex_yes_phase_added.evt


In [32]:
hdul=fits.open('/home/supremekai/thesis/J0243_6p6124/nicer/reduced_output/6050390231/ni6050390231_0mpu7_cl_night_barycorr_orbit_piexpiex_yes_trial_phase_resolved_test.evt')

hdul.info()

Filename: /home/supremekai/thesis/J0243_6p6124/nicer/reduced_output/6050390231/ni6050390231_0mpu7_cl_night_barycorr_orbit_piexpiex_yes_trial_phase_resolved_test.evt
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      32   ()      
  1  EVENTS        1 BinTableHDU    441   1657510R x 17C   [1D, 1B, 1B, 1I, 1I, 1B, 1B, 8X, 1K, I, J, 1I, 1I, 1E, 1D, 1D, 1D]   
  2  FPM_SEL       1 BinTableHDU    128   2487R x 3C   [1D, 56B, 56I]   
  3  GTI           1 BinTableHDU    258   5R x 2C   [D, D]   
  4  GTI_DET10     1 BinTableHDU    224   4R x 2C   [D, D]   
  5  GTI_DET21     1 BinTableHDU    225   2R x 2C   [D, D]   
  6  GTI_DET42     1 BinTableHDU    225   37R x 2C   [D, D]   
  7  GTI_DET43     1 BinTableHDU    225   3R x 2C   [D, D]   
  8  GTI_MPU0      1 BinTableHDU    264   24R x 2C   [D, D]   
  9  GTI_MPU1      1 BinTableHDU    257   38R x 2C   [D, D]   
 10  GTI_MPU2      1 BinTableHDU    261   37R x 2C   [D, D]   
 11  GTI_MPU3      1

In [33]:
events=hdul[1]

In [34]:
events.columns

ColDefs(
    name = 'TIME'; format = '1D'; unit = 's'; disp = 'F20.9'
    name = 'RAWX'; format = '1B'; unit = 'pixel'; coord_type = 'RAWX'; coord_ref_point = 0; coord_ref_value = 0.0; coord_inc = 1.0
    name = 'RAWY'; format = '1B'; unit = 'pixel'; coord_type = 'RAWY'; coord_ref_point = 0; coord_ref_value = 0.0; coord_inc = 1.0
    name = 'PHA'; format = '1I'; unit = 'chan'; null = -32768
    name = 'PHA_FAST'; format = '1I'; unit = 'chan'; null = -32768
    name = 'DET_ID'; format = '1B'; disp = 'I2.2'
    name = 'DEADTIME'; format = '1B'; unit = 's'; bscale = 6.202e-07; bzero = 3.101e-07; disp = 'F11.9'
    name = 'EVENT_FLAGS'; format = '8X'
    name = 'TICK'; format = '1K'
    name = 'MPU_A_TEMP'; format = 'I'; unit = 'Celsius'; null = -1; bscale = 0.10600843; bzero = -273.42105; disp = 'F6.3'
    name = 'MPU_UNDER_COUNT'; format = 'J'; null = -1
    name = 'PI_FAST'; format = '1I'; unit = 'chan'; null = -32768
    name = 'PI'; format = '1I'; unit = 'chan'; null = -32768
    name

In [4]:
phase=events.data['PHASE']

In [5]:
phase.size

1657510